In [163]:
# import libraries
# DataFrame
import pandas as pd

# plotting
import seaborn as sns
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# nltk
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# sklearny
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB


#tensorflow
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Utility
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import re
import string
import pickle
# RNN - LSTM
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import re
import keras
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import Adam

# Test Input dataset before model training

In [164]:
nltk.download('stopwords')
stopword = set(stopwords.words('english'))
print(stopword)

{'more', 'ain', 'such', 'was', 'above', 'at', 'before', 'where', "aren't", 'have', "we're", "she'd", 've', 'between', 'wasn', 'there', 't', 'that', 'both', 'so', 'a', 'no', "they'd", 'how', 'those', "don't", 'what', 'our', "it'd", "should've", 'having', 'ma', 'or', "we'll", 'during', 'will', 'm', 'won', 'down', "weren't", 'll', "it's", 'most', 'same', "isn't", 'o', "we've", 'about', 'couldn', 'while', 'under', 'after', "that'll", 'themselves', 'up', 'am', "mightn't", 'nor', "wasn't", 'why', 'ourselves', 'mightn', 'd', 'until', 'these', "hadn't", 'is', 'did', 'now', 'are', 'than', 'be', "doesn't", 'been', 'his', 'me', 'she', 'don', 'their', "i'm", 'further', 'other', 'then', "didn't", 'not', 'can', "you'd", 'only', 'of', 'for', 'all', 'he', 'off', 'needn', 'doesn', 'does', 'into', 'each', 'i', "they're", 'whom', 'herself', "she'll", 'we', "shouldn't", 'him', 'once', "hasn't", 'out', 'on', 'had', 'weren', 'ours', 'below', 'myself', "couldn't", "shan't", "needn't", 'my', 'against', 'would

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zhaijingjing/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [165]:
urlPattern = r"((http://)[^ ]*|(https://)[^ ]*|( www\.)[^ ]*)"
userPattern = '@[^\s]+'
def process_tweets(tweet):
  # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        finalwords.append(word)
    return ' '.join(finalwords)

def process_tweets_append(tweet):
  # Lower Casing
    tweet = tweet.lower()
    #tweet=tweet[1:]
    # Removing all URls
    tweet = re.sub(urlPattern,'',tweet)
    # Removing all @username.
    tweet = re.sub(userPattern,'', tweet)
    #Remove punctuations
    tweet = tweet.translate(str.maketrans("","",string.punctuation))
    #tokenizing words
    tokens = word_tokenize(tweet)
    #Removing Stop Words
    #final_tokens = [w for w in tokens if w not in stopword]
    final_tokens = tokens
    #reducing a word to its word stem
    wordLemm = WordNetLemmatizer()
    finalwords=[]
    negation_words = {"not", "no", "never", "n't"}
    prev_neg = False

    for w in final_tokens:
      if len(w)>1:
        word = wordLemm.lemmatize(w)
        #finalwords.append(word)
        if word in negation_words:
          prev_neg = True
        elif prev_neg:
          finalwords.append("not_" + word)
          prev_neg = False
        else:
          finalwords.append(word)
    return ' '.join(finalwords)

In [166]:
def tokenizer(max_words, test_text):
    # 1. build word table with top 50 words
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(test_text)
    print("单词表:", tokenizer.word_index)

    sequences = tokenizer.texts_to_sequences(test_text)
    print("文本的单词索引序列:", sequences)

    tweets = pad_sequences(sequences, maxlen=10)
    print(tweets.shape)
    print("填充后文本的单词索引序列:", tweets)

In [167]:
# Test
max_words = 20
test_tweets = [
    "I LOVE the movie!!!@user123",
    "Not happy with the service at all",
    "Never coming back! website at https://www.example.com",
    "Not happy with the service at all."
]

processed_test_text = [process_tweets_append(x) for x in test_tweets]
print("预处理后的数据:", processed_test_text)
tokenizer(max_words, processed_test_text)

预处理后的数据: ['love the movie', 'not_happy with the service at all', 'not_coming back website at', 'not_happy with the service at all']
单词表: {'the': 1, 'not': 2, 'at': 3, 'happy': 4, 'with': 5, 'service': 6, 'all': 7, 'love': 8, 'movie': 9, 'coming': 10, 'back': 11, 'website': 12}
文本的单词索引序列: [[8, 1, 9], [2, 4, 5, 1, 6, 3, 7], [2, 10, 11, 12, 3], [2, 4, 5, 1, 6, 3, 7]]
(4, 10)
填充后文本的单词索引序列: [[ 0  0  0  0  0  0  0  8  1  9]
 [ 0  0  0  2  4  5  1  6  3  7]
 [ 0  0  0  0  0  2 10 11 12  3]
 [ 0  0  0  2  4  5  1  6  3  7]]
